In [1]:
import pandas as pd

train_df = pd.read_csv("train_preprocessed_columns_dropped.csv")

train_df.head()

,tanggal_pencairan,jumlah_pinjaman,total_pengembalian,durasi_hari,porsi_pengembalian_lender,status_gagal_bayar,provinsi_freq,jenis_pinjaman_freq,status_peminjam_freq,sektor_usaha_freq,pendidikan_freq,jenis_jaminan_freq,provinsi_te,jenis_pinjaman_te,status_peminjam_te,sektor_usaha_te,pendidikan_te,jenis_jaminan_te
0,8613,959000.0,959000.0,7.0,287000.0,0,0.254831,0.903050,0.992441,0.166294,0.247272,0.248850,0.046624,0.023930,0.041643,0.036650,0.044221,0.044751
1,8624,55000.0,55000.0,7.0,16000.0,0,0.253714,0.903050,0.992441,0.166097,0.052715,0.244249,0.044285,0.024852,0.043126,0.044456,0.034125,0.039099
2,8617,655000.0,666000.0,7.0,199000.0,0,0.254831,0.903050,0.992441,0.166294,0.400684,0.248850,0.048168,0.023467,0.041901,0.037096,0.043523,0.044561
3,9432,611000.0,632000.0,7.0,126000.0,0,0.253714,0.046602,0.992441,0.166097,0.299330,0.253977,0.043520,0.186367,0.041643,0.044110,0.044032,0.040323
4,8652,573000.0,573000.0,7.0,127000.0,0,0.241159,0.903050,0.992441,0.158275,0.052715,0.252925,0.044308,0.024852,0.043126,0.045267,0.034125,0.050668


In [2]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15214 entries, 0 to 15213
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tanggal_pencairan          15214 non-null  int64  
 1   jumlah_pinjaman            15214 non-null  float64
 2   total_pengembalian         15214 non-null  float64
 3   durasi_hari                15214 non-null  float64
 4   porsi_pengembalian_lender  15214 non-null  float64
 5   status_gagal_bayar         15214 non-null  int64  
 6   provinsi_freq              15214 non-null  float64
 7   jenis_pinjaman_freq        15214 non-null  float64
 8   status_peminjam_freq       15214 non-null  float64
 9   sektor_usaha_freq          15214 non-null  float64
 10  pendidikan_freq            15214 non-null  float64
 11  jenis_jaminan_freq         15214 non-null  float64
 12  provinsi_te                15214 non-null  float64
 13  jenis_pinjaman_te          15214 non-null  flo

In [3]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = train_df[train_df.status_gagal_bayar == 0]
df_minority = train_df[train_df.status_gagal_bayar == 1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                               replace=True,     # sample with replacement
                               n_samples=len(df_majority),    # to match majority class
                               random_state=42) # reproducible results

# Combine majority class with upsampled minority class
train_df = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
print(train_df.status_gagal_bayar.value_counts())

x = train_df.drop(columns=['status_gagal_bayar'])
y = train_df['status_gagal_bayar']

status_gagal_bayar
0    14539
1    14539
Name: count, dtype: int64


In [4]:
from sklearn.model_selection import train_test_split

# Split for validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

In [6]:
from catboost import CatBoostClassifier

# Train model
xgb_model = CatBoostClassifier(random_state=42, verbose=0)
xgb_model.fit(x_train, y_train)

# Validation
y_pred = xgb_model.predict(x_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      2885
           1       0.98      1.00      0.99      2931

    accuracy                           0.99      5816
   macro avg       0.99      0.99      0.99      5816
weighted avg       0.99      0.99      0.99      5816



In [7]:
df = pd.read_csv("test_preprocessed_with_freq_te_and_columns_dropped.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5003 entries, 0 to 5002
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id_transaksi               5003 non-null   int64  
 1   tanggal_pencairan          5003 non-null   int64  
 2   jumlah_pinjaman            5003 non-null   float64
 3   total_pengembalian         4946 non-null   float64
 4   durasi_hari                4853 non-null   float64
 5   porsi_pengembalian_lender  4957 non-null   float64
 6   provinsi_freq              5003 non-null   float64
 7   jenis_pinjaman_freq        5003 non-null   float64
 8   status_peminjam_freq       5003 non-null   float64
 9   sektor_usaha_freq          5003 non-null   float64
 10  pendidikan_freq            5003 non-null   float64
 11  jenis_jaminan_freq         5003 non-null   float64
 12  provinsi_te                5003 non-null   float64
 13  jenis_pinjaman_te          5003 non-null   float

In [35]:
df = pd.read_csv("test_preprocessed_with_freq_te_and_columns_dropped.csv")

test_preds = xgb_model.predict(df.drop(columns=['id_transaksi']))

# Tambahkan ke dataframe
df['status_gagal_bayar'] = test_preds

# Drop every column except 'id_transaksi' and 'status_gagal_bayar'
df = df[['id_transaksi', 'status_gagal_bayar']]

CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:81: At position 1 should be feature with name jenis_pinjaman (found jumlah_pinjaman).

In [ ]:
df.head()

,id_transaksi,status_gagal_bayar
0,20013,0
1,20014,0
2,20015,0
3,20016,0
4,20017,0


In [ ]:
df.to_csv("submission_catboost_impute median with columns dropped and oversampling.csv", index=False)